In [1]:
import os
import shutil
import time

from environment import StreetFighterEnv
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.callbacks import BaseCallback

import optuna
from model import policy_kwargs
import warnings

warnings.simplefilter("ignore")

In [2]:
def optimize_ppo(trial):
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [3]:
BASE_DIR = './A2C_without_attention_frame_4/'
LOG_DIR = '{}logs/'.format(BASE_DIR)
OPT_DIR = '{}opt/'.format(BASE_DIR)
CHECKPOINT_DIR = '{}train/'.format(BASE_DIR)
TRAIN_LOG_DIR = '{}train_logs/'.format(BASE_DIR)
RECORD_DIR = '{}record/'.format(BASE_DIR)

for i in [BASE_DIR, LOG_DIR, OPT_DIR, CHECKPOINT_DIR, TRAIN_LOG_DIR, RECORD_DIR]:

    shutil.rmtree(i, ignore_errors=True)
    os.mkdir(i)
    
    
def optimize_agent(trial):

    model_params = optimize_ppo(trial)

    print(model_params)
    env = StreetFighterEnv(RECORD_DIR)
    env = Monitor(env, LOG_DIR)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env, 4, channels_order='last')
    model = A2C("CnnPolicy", env, verbose=0, tensorboard_log=LOG_DIR, policy_kwargs=policy_kwargs, **model_params)

    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
    model.learn(total_timesteps=100000)
    env.close()

    SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
    model.save(SAVE_PATH)

    return mean_reward
  

In [4]:
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent, n_trials=20, n_jobs=1)


[I 2022-04-14 12:50:24,044] A new study created in memory with name: no-name-4997c5d4-0040-4543-a570-a1e8186c96c5


{'n_steps': 7748, 'gamma': 0.8670080821618195, 'learning_rate': 7.481195328501302e-05, 'gae_lambda': 0.8073508792310118}


[W 2022-04-14 12:52:44,220] Trial 0 failed because of the following error: RuntimeError('CUDA error: out of memory')
Traceback (most recent call last):
  File "/home/ashutosh/miniconda3/lib/python3.8/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-3-a662cbbdb411>", line 26, in optimize_agent
    model.learn(total_timesteps=100000)
  File "/home/ashutosh/miniconda3/lib/python3.8/site-packages/stable_baselines3/a2c/a2c.py", line 191, in learn
    return super(A2C, self).learn(
  File "/home/ashutosh/miniconda3/lib/python3.8/site-packages/stable_baselines3/common/on_policy_algorithm.py", line 250, in learn
    continue_training = self.collect_rollouts(self.env, callback, self.rollout_buffer, n_rollout_steps=self.n_steps)
  File "/home/ashutosh/miniconda3/lib/python3.8/site-packages/stable_baselines3/common/on_policy_algorithm.py", line 168, in collect_rollouts
    obs_tensor = obs_as_tensor(self._last_obs, self.dev

RuntimeError: CUDA error: out of memory

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['gamma', 'learning_rate', 'n_steps'])

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_edf(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
study.best_params

In [ ]:
path = os.path.join(OPT_DIR, 'trial_{}_best_model.zip'.format(study.best_trial.number))
print(path)
model = A2C.load(path)

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
# for i in [LOG_DIR, CHECKPOINT_DIR, TRAIN_LOG_DIR, RECORD_DIR]:
        
#     shutil.rmtree(i)
#     os.mkdir(i)
    
model_params = study.best_params
model_params['n_steps'] = 7488

env = StreetFighterEnv(RECORD_DIR)
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')
model = A2C("CnnPolicy", env, verbose=1, tensorboard_log=LOG_DIR, policy_kwargs=policy_kwargs, **model_params)


In [ ]:
model.load(path)

In [ ]:
model.learn(total_timesteps=30000, callback=callback)

In [ ]:
mean_reward, _ = evaluate_policy(model, env, render=False, n_eval_episodes=5)
mean_reward

In [ ]:
model.save(BASE_DIR + 'final_model')

In [ ]:
# obs = env.reset()
# obs.shape

In [ ]:
# env.step(model.predict(obs)[0])

In [ ]:
# # Reset game to starting state
# obs = env.reset()
# # Set flag to flase
# done = False
# for game in range(1):
#     while not done:
#         if done:
#             obs = env.reset()
#         env.render()
#         action = model.predict(obs)[0]
#         obs, reward, done, info = env.step(action)
#         time.sleep(0.01)
#         if reward: print(reward)